<p style="font-weight:bold;"> <span style="font-size: 36px"> IFRS17 Calculation Engine </span> </p>

In [ ]:
#!import "../ifrs17/Import/Importers"

In [ ]:
// TODO: avoid scopes with the same name, e.g. Csm, Lc
#!import "../ifrs17/Report/ReportScopes"

In [ ]:
#!import "../ifrs17/Report/ReportConfigurationAndUtils"

In [ ]:
#r "nuget:Systemorph.Charting,1.3.0"

# TMP replacement methods

In [ ]:
// TODO: improve the way we get identities, not related to this project 
//public static ImportIdentity GetIdentity (this Universe universe, string dataNode, string aocType, string novelty) 
//    => universe.GetScope<GetIdentities>((dataNode, (int?)null)).Identities.Single( x => x.AocType == aocType && x.Novelty == novelty );

In [ ]:
// TODO: check whether to replace the existing method "GetElementOrDefault"
static T GetValidElement<T>(this IList<T> array, int index) => GetElementOrDefault(array, index);
static T GetValidElement<T>(this IEnumerable<T> array, int index) => GetElementOrDefault(array.ToArray(), index);

In [ ]:
public static IDataCube<RawVariable> ComputeDiscountAndCumulate ( this IDataCube<RawVariable> nominals, double[] yearlyDiscountRates, AmountType[] amountTypes ) 
{
    if(nominals == null) return Enumerable.Empty<RawVariable>().ToDataCube();
       
    return nominals.Select(x => {
        var nominal = x.Values.ToArray();
        var cdcf = new double[nominal.Length];
        var period = amountTypes.FirstOrDefault(a => a.SystemName == x.AmountType)?.PeriodType?? PeriodType.EndOfPeriod;

        if(period == PeriodType.BeginningOfPeriod)
        {
            for (var i = cdcf.Length - 1; i >= 0; i--)
                cdcf[i] = nominal[i] + GetElementOrDefault(cdcf, i + 1) * GetElementOrDefault(yearlyDiscountRates, i/12);
        }
        else
        { 
            for (var i = cdcf.Length - 1; i >= 0; i--)
                cdcf[i] = ( nominal[i] + GetElementOrDefault(cdcf, i + 1) ) * GetElementOrDefault(yearlyDiscountRates, i/12);
        }
        return x with { Values = cdcf };
    })
    .ToDataCube();
}

# TMP report configurations

In [ ]:
// TODO: this must be improved and included in the api
const string ArrayFormatter = "value.map(v => new Intl.NumberFormat([], { maximumFractionDigits: 8 }).format(v)).join(' , ')";

In [ ]:
// TODO: the default must include this 
public static ReportBuilder<YieldCurve,YieldCurve,YieldCurve> WithGridOptionsForYieldCurves (this ReportBuilder<YieldCurve,YieldCurve,YieldCurve> reportBuilder, int reportHeight = 200)
{
    return reportBuilder.WithGridOptions(o => o.WithColumns(c => c.Modify("Values", x => x with { ValueFormatter = ArrayFormatter, CellStyle = new { TextAlign = "left" } } ))
                                         with { Height = reportHeight, TreeData = false } );  // AutoGroupColumnDef = null (what is this?)
}

In [ ]:
//Report.ForObjects(yieldCurves).WithGridOptionsForYieldCurves().WithQuerySource(DataSource).ToReport()

In [ ]:
using System.Collections.Immutable;
// TODO: The type or namespace name 'ToImmutableList' does not exist in the namespace 'System.Collections.Immutable
//System.Collections.Immutable.ToImmutableList(r.RowGroup.Coordinates.Where(c => c != "NullGroup")) 

// TODO: discuss default here too, especially the trick to remove empty row
public static ReportBuilder<RawVariable,RawVariable,RawVariable> WithGridOptionsForCashflows (this ReportBuilder<RawVariable,RawVariable,RawVariable> reportBuilder, int reportHeight = 450)
{
    return reportBuilder.GroupColumnsBy(x => x.DataNode)
                        //.GroupColumnsBy(x => x.CalculationType)
                        .GroupRowsBy(x => x.AmountType)
                        .GroupRowsBy(x => x.AocType)
                        .WithGridOptions(o => o.WithRows(r => r.Where(r => !(r.RowGroup.Coordinates.Last() == "NullGroup"))
                                                               .Select(r => r with { RowGroup = r.RowGroup with { Coordinates = r.RowGroup.Coordinates.Where(c => c != "NullGroup").ToImmutableList() } }).ToList())
                                               .WithColumns(c => c.Modify("Values", x => x with { ValueFormatter = ArrayFormatter, CellStyle = new { TextAlign = "left" } } ))
                                         with { Height = reportHeight, GroupDefaultExpanded = 2 } );
}

In [ ]:
public static DataCubeReportBuilder<IDataCube<RawVariable>, RawVariable, RawVariable ,RawVariable> WithGridOptionsForCashflows 
(this DataCubeReportBuilder<IDataCube<RawVariable>, RawVariable, RawVariable ,RawVariable> reportBuilder, int reportHeight = 450)
{
    return reportBuilder.SliceColumnsBy(nameof(GroupOfContract))//,nameof(CalculationType))
                        .SliceRowsBy(nameof(AmountType),nameof(AocType))
                        .WithGridOptions(o => o.WithRows(r => r.Where(r => !(r.RowGroup.Coordinates.Last() == "NullGroup"))
                                                               .Select(r => r with { RowGroup = r.RowGroup with { Coordinates = r.RowGroup.Coordinates.Where(c => c != "NullGroup").ToImmutableList() } }).ToList())
                                               .WithColumns(c => c.Modify("Values", x => x with { ValueFormatter = ArrayFormatter, CellStyle = new { TextAlign = "left" } } ))
                                         with { Height = reportHeight, GroupDefaultExpanded = 2 } );
}

In [ ]:
//Report.ForDataCube(nominals).WithGridOptionsForCashflows(500).WithQuerySource(DataSource).ToReport()

# TMP report api

In [ ]:
[InitializeScope(nameof(InitAsync))]
public interface PvReportTmp: IMutableScopeWithStorage<ReportStorage> {
    // Some infrastructure
    private IWorkspace workspace => GetStorage().Workspace;
    private Systemorph.Vertex.Pivot.Builder.Interfaces.IPivotFactory report => GetStorage().Report;
    
    // Basic mutable properties
    (int Year, int Month) ReportingPeriod { get; set; }
    int Year => ReportingPeriod.Year;
    int Month => ReportingPeriod.Month;
    string ReportingNode { get; set; }
    string Scenario { get; set; }
    CurrencyType CurrencyType { get; set; }
    
    ((int Year, int Month) ReportingPeriod, string ReportingNode, string Scenario, CurrencyType) ShowSettings => (ReportingPeriod, ReportingNode, Scenario, CurrencyType);
    
    // Slice and Dice
    IEnumerable<string> RowSlices { get; set; }
    private string[] defaultRowSlices => new string[] { "Novelty", "VariableType" };
    private string[] rowSlices => RowSlices is null ? defaultRowSlices : defaultRowSlices.Concat(RowSlices).ToArray();
    
    IEnumerable<string> ColumnSlices { get; set; }
    // TODO: include EconomicBasis in the default slices
    private string[] defaultColumnSlices => new string[] { "ReportingNode", CurrencyGrouper(CurrencyType), "EstimateType", "EconomicBasis" };
    private string[] columnSlices => ColumnSlices is null ? defaultColumnSlices : defaultColumnSlices.Concat(ColumnSlices).ToArray();
    
    // Filter
    // TODO: this is bugged, since the default null will remove all filters
    IEnumerable<(string filterName, string filterValue)> DataFilter { get; set; }
    private (string filterName, object filterValue)[] dataFilter => (DataFilter is null ? Enumerable.Empty<(string, object)>() : DataFilter.Select(x => (x.filterName, (object)x.filterValue))).ToArray();
    
    // Scope Initialization
    async Task InitAsync() {
        var mostRecentPartition = (await workspace.Query<PartitionByReportingNodeAndPeriod>().Where(x => x.Scenario == null).OrderBy(x => x.Year).ThenBy(x => x.Month).ToArrayAsync()).Last();  
        
        ReportingPeriod = (mostRecentPartition.Year, mostRecentPartition.Month);
        ReportingNode = (await workspace.Query<ReportingNode>().Where(x => x.Parent == null).ToArrayAsync()).First().SystemName;;
        Scenario = null;
        CurrencyType = CurrencyType.Contractual;       
        ColumnSlices = new[] {"AmountType"};
        await GetStorage().InitializeReportIndependentCacheAsync();
    }
    
    // Data update in Storage and Report Generation
    async Task<GridOptions> ToReportAsync() {
        await GetStorage().InitializeAsync(ReportingPeriod, ReportingNode, Scenario, CurrencyType);
    
        var identities = GetStorage().GetIdentities(ReportingPeriod, ReportingNode, Scenario, CurrencyType);
        var dataCube = GetScopes<LockedBestEstimate>(identities).Aggregate().LockedBestEstimate;
        
        return report.ForDataCube(dataCube)
                     .WithQuerySource(workspace)
                     .SliceRowsBy(rowSlices)
                     .SliceColumnsBy(columnSlices)
                     .ReportGridOptions(reportHeight : 600)
                     .ToReport();
    }
    
    // TODO: return a ReportBuilder
    async Task<DataCubeReportBuilder<IDataCube<ReportVariable>, ReportVariable, ReportVariable, ReportVariable>> ToReportBuilderAsync() {
        await GetStorage().InitializeAsync(ReportingPeriod, ReportingNode, Scenario, CurrencyType);
    
        var identities = GetStorage().GetIdentities(ReportingPeriod, ReportingNode, Scenario, CurrencyType);
        var dataCube = (GetScopes<LockedBestEstimate>(identities).Aggregate().LockedBestEstimate + GetScopes<CurrentBestEstimate>(identities).Aggregate().CurrentBestEstimate).Filter(dataFilter);
        
        return report.ForDataCube(dataCube)
            .WithQuerySource(workspace)
            .SliceRowsBy(rowSlices)
            .SliceColumnsBy(columnSlices)
            .ReportGridOptions();
    }
}

In [ ]:
// TODO: move it to calculation engine
public class Ifrs17 
{
    private IWorkspace Workspace {get; init;}
    private Systemorph.Vertex.Scopes.Proxy.IScopeFactory Scopes {get; init;}
    private Systemorph.Vertex.Pivot.Builder.Interfaces.IPivotFactory Report {get; init;}
    private ReportStorage Storage {get; set;}
    
    public PvReportTmp PresentValues {get{
        Storage = new ReportStorage(Workspace,Report);
        return Scopes.ForSingleton().WithStorage(Storage).ToScope<PvReportTmp>();
    }}
    
    // TODO: temporary just for the feeling of having many topics to present
    public PvReportTmp RiskAdjustment {get{
        Storage = new ReportStorage(Workspace,Report);
        return Scopes.ForSingleton().WithStorage(Storage).ToScope<PvReportTmp>();
    }}
    
    // TODO: temporary just for the feeling of having many topics to present
    public PvReportTmp ContractualServiceMargin {get{
        Storage = new ReportStorage(Workspace,Report);
        return Scopes.ForSingleton().WithStorage(Storage).ToScope<PvReportTmp>();
    }}
    
    // TODO: temporary just for the feeling of having many topics to present
    public PvReportTmp FinancialPerformance {get{
        Storage = new ReportStorage(Workspace,Report);
        return Scopes.ForSingleton().WithStorage(Storage).ToScope<PvReportTmp>();
    }}
    
    public Ifrs17 (IWorkspace ws, Systemorph.Vertex.Scopes.Proxy.IScopeFactory scopes, Systemorph.Vertex.Pivot.Builder.Interfaces.IPivotFactory report)
    {
        Workspace = ws; 
        Scopes = scopes; 
        Report = report; 
    }
}

In [ ]:
var Ifrs17 = new Ifrs17(Workspace,Scopes,Report);

# TMP Waterfall Chart

var identities = pv.GetStorage().GetIdentities((2021,3), "G", null);
var datacube = pv.GetScopes<LockedBestEstimate>(identities).Aggregate().LockedBestEstimate;
var pr = datacube.Filter(("AmountType","PR"));
var cl = datacube.Filter(("AmountType","CL"));

double[] pr1 = {0,
                pr.Filter(("VariableType","BOP")).First().Value,
                pr.Filter(("VariableType","BOP")).First().Value
                    +pr.Filter(("VariableType","CF")).First().Value,
                pr.Filter(("VariableType","BOP")).First().Value
                    +pr.Filter(("VariableType","CF")).First().Value
                    +pr.Filter(("VariableType","IA")).First().Value,
                pr.Filter(("VariableType","BOP")).First().Value
                    +pr.Filter(("VariableType","CF")).First().Value
                    +pr.Filter(("VariableType","IA")).First().Value
                    +pr.Filter(("VariableType","AU")).First().Value,
                0};
double[] pr2 = {pr.Filter(("VariableType","BOP")).First().Value,
                pr.Filter(("VariableType","BOP")).First().Value
                    +pr.Filter(("VariableType","CF")).First().Value,
                pr.Filter(("VariableType","BOP")).First().Value
                    +pr.Filter(("VariableType","CF")).First().Value
                    +pr.Filter(("VariableType","IA")).First().Value,
                pr.Filter(("VariableType","BOP")).First().Value
                    +pr.Filter(("VariableType","CF")).First().Value
                    +pr.Filter(("VariableType","IA")).First().Value
                    +pr.Filter(("VariableType","AU")).First().Value,
                pr.Filter(("VariableType","BOP")).First().Value
                    +pr.Filter(("VariableType","CF")).First().Value
                    +pr.Filter(("VariableType","IA")).First().Value
                    +pr.Filter(("VariableType","AU")).First().Value
                    +pr.Filter(("VariableType","EV")).First().Value,
                pr.Filter(("VariableType","EOP")).First().Value};

double[] cl1 = {0,
                cl.Filter(("VariableType","BOP")).First().Value,
                cl.Filter(("VariableType","BOP")).First().Value
                    +cl.Filter(("VariableType","CF")).First().Value,
                cl.Filter(("VariableType","BOP")).First().Value
                    +cl.Filter(("VariableType","CF")).First().Value
                    +cl.Filter(("VariableType","IA")).First().Value,
                cl.Filter(("VariableType","BOP")).First().Value
                    +cl.Filter(("VariableType","CF")).First().Value
                    +cl.Filter(("VariableType","IA")).First().Value
                    +cl.Filter(("VariableType","AU")).First().Value,
                0};
double[] cl2 = {cl.Filter(("VariableType","BOP")).First().Value,
                cl.Filter(("VariableType","BOP")).First().Value
                    +cl.Filter(("VariableType","CF")).First().Value,
                cl.Filter(("VariableType","BOP")).First().Value
                    +cl.Filter(("VariableType","CF")).First().Value
                    +cl.Filter(("VariableType","IA")).First().Value,
                cl.Filter(("VariableType","BOP")).First().Value
                    +cl.Filter(("VariableType","CF")).First().Value
                    +cl.Filter(("VariableType","IA")).First().Value
                    +cl.Filter(("VariableType","AU")).First().Value,
                cl.Filter(("VariableType","BOP")).First().Value
                    +cl.Filter(("VariableType","CF")).First().Value
                    +cl.Filter(("VariableType","IA")).First().Value
                    +cl.Filter(("VariableType","AU")).First().Value
                    +cl.Filter(("VariableType","EV")).First().Value,
                cl.Filter(("VariableType","EOP")).First().Value};

string[] aocs = {"BOP", "CF", "IA", "AU", "EV", "EOP"};

string[] labels = {"Opening Balance", "Cash flow", "Interest Accretion", "Assumption Update", "Experience Variance", "Closing Balance"};

// TODO: talk to A.Kravets 
Chart.FloatingBar()
     .WithDataRange(pr1, pr2, options => options.WithLabel("Premiums"))
     .WithDataRange(cl1, cl2, options => options.WithLabel("Claims"))
     .WithLabels(labels)
     .WithTitle("Analysis of Change")

# TMP Report Chart

In [ ]:
ChartBuilder.GetType()

In [ ]:
static Systemorph.Vertex.Charting.Builders.ChartBuilderVariable chart;
chart = (Systemorph.Vertex.Charting.Builders.ChartBuilderVariable)ChartBuilder;

public static YieldCurve[] ForObject
(this Systemorph.Vertex.Pivot.Builder.Interfaces.IPivotFactory report, YieldCurve[] data) 
    => data;

public static Systemorph.Vertex.DataCubes.DataCube<RawVariable> ForDatacube
(this Systemorph.Vertex.Pivot.Builder.Interfaces.IPivotFactory report, Systemorph.Vertex.DataCubes.Api.IDataCube<RawVariable> data) 
    => (Systemorph.Vertex.DataCubes.DataCube<RawVariable>)data;

public static Systemorph.Vertex.Charting.Builders.ChartBuilders.LineChartBuilder WithOptionsForEconomicRates 
(this YieldCurve[] ycs)
{
    var res = chart.Line();
    var max = -100.0;
    var min = +100.0;
    foreach(var yc in ycs) 
    {
        var interest = (from x in yc.Values select Math.Pow((1 + x),( +1.0 / 12.0 ))).ToArray();
        //var discount = (from x in yc.Values select (1 + x)^( -1.0 / 12.0 )).ToArray();
        if(interest.Max() > max) max = interest.Max();
        //if(discount.Max() > max) max = discount.Max();
        if(interest.Min() < min) min = interest.Min();
        //if(discount.Min() < min) min = discount.Min();
        res.WithDataSet( o => o.WithData(interest).WithLabel( yc.Currency) );
        //.WithDataArray( discount , o => o.WithLabel("Discount " + yc.Currency) );
    }
    return res.WithOptions(o => o.WithYAxisMin(Math.Truncate((min-0.001) * 1000) / 1000))
              .WithOptions(o => o.WithYAxisMax(Math.Truncate((max+0.001) * 1000) / 1000))
              .WithTitle("Discount Factor")
              .WithLegend();
}

public static Systemorph.Vertex.Charting.Builders.ChartBuilders.BarChartBuilder WithOptionsForNominalCashFlows
(this Systemorph.Vertex.DataCubes.DataCube<RawVariable> cfs)
{
    return chart.Bar().WithTitle("Nominal Cashflows", o => o.WithFontSize(20))
                      .WithDataSet(o => o.WithData(cfs.First(x => x.AmountType == "PR" && x.AocType == "BOP") .Values).WithLabel("Premiums BOP"))
                      .WithDataSet(o => o.WithData(cfs.First(x => x.AmountType == "PR" && x.AocType == "AU")  .Values).WithLabel("Premiums AU"))
                      .WithDataSet(o => o.WithData(cfs.First(x => x.AmountType == "PR" && x.AocType == "EV")  .Values).WithLabel("Premiums EV"))
                      .WithDataSet(o => o.WithData(cfs.First(x => x.AmountType == "PR" && x.AocType == "CL")  .Values).WithLabel("Premiums CL"))
                      .WithDataSet(o => o.WithData(cfs.First(x => x.AmountType == "CL" && x.AocType == "BOP") .Values).WithLabel("Claims BOP"))
                      .WithDataSet(o => o.WithData(cfs.First(x => x.AmountType == "CL" && x.AocType == "EV")  .Values).WithLabel("Claims EV"))
                      .WithDataSet(o => o.WithData(cfs.First(x => x.AmountType == "CL" && x.AocType == "CL")  .Values).WithLabel("Claims CL"))
                      .WithLegend();
}

public static Systemorph.Vertex.Charting.ChartPresenterData ToLineChart
(this Systemorph.Vertex.Charting.Builders.ChartBuilders.LineChartBuilder charting) 
    => charting.ToChart();

public static Systemorph.Vertex.Charting.ChartPresenterData ToBarChart
(this Systemorph.Vertex.Charting.Builders.ChartBuilders.BarChartBuilder charting) 
    => charting.ToChart();

.

.

.

.

.

# TMP How to make it nicer

        RESTRICT THE FOCUS OF THIS NOTEBOOK (keep 1 instead of n major subjects)

- **Parameters** to be changed by the user --> Yield Curve, Cashflows (!)

- **Insurance** or Reinsurance or both --> just the first (!)

- **no exchange rates** for the moment (big topic, depends on period type), better to have one separate notebook dealing with multi-currencies (!)

- **irreducible data set which tells the entire story**

    - reduce number of Amount Types, e.g. keep only the roots (Premiums, Claims, Attributable Expenses, Non-attributable Expenses, Attributable Commission)

# TMP Limitations of the current IFRS17 engine

        LIMITATIONS (...)

- Granularity of the **yield curve**: at the moment it is yiearly

- The **yield curve** is (trivially) extrapolated, that is, the last element is considered 

- Granularity of the cashflows: at the moment it is monthly (hardcoded, see 1/12 in yield curve)

- At the moment no warning is thrown when appropriate Yield Curve is not found for the target period